Data from: http://zhao-nas.bio.cmu.edu:5000/fsdownload/aBDx29J7H/Ensemble%20learning%20data_shared

In [1]:
# import os
# if hasattr(os, 'add_dll_directory'):
#     openslide_dir = r"C:\Users\Windows\Downloads\openslide-win64-20220811\bin"
#     print(f"Adding openslide dir at:{openslide_dir}")
#     with os.add_dll_directory(openslide_dir):
#         import openslide
# else:
#     import openslide
# from tiatoolbox.tools import patchextraction
# from tiatoolbox.utils.misc import imread,  read_locations
# from tiatoolbox.tools import stainnorm
# from tiatoolbox import data
# from tiatoolbox.wsicore.wsireader import WSIReader

In [2]:
from pathlib import Path
import cv2 , os, numpy as np, torch, pandas as pd, tqdm as tqdm, PIL.Image as Image, time
#from pylab import rcParams
import datetime
# 
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torchsummary import summary
from pytorch_lightning.loops.fit_loop import FitLoop
from pytorch_lightning.loops.loop import Loop
#
from numpy.lib.function_base import select
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#
#import efficientnet_pytorch
import pytorch_lightning as pl, torchmetrics

/home/windows/anaconda3/envs/vision/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prefix=Path('/mnt/c/data/MattM_Ensemble')

class PATH_ARGS:
    proj_path = Path('./').absolute()  # [CHANGE THIS for new environment]
    model_path = proj_path/'model_chkpts'
    # data path
    #data_path = proj_path/'TestingData'   # Test path
    #data_path = proj_path/"Ensemble_learning data"      # [CONFIRM THIS for new environment]
    data_path = proj_path.parent
    data_name = ['HE_RBG_Corp_images']
    dataindex_path = data_path/'dataIndex.csv'
    # Select data folders here
    #data_name = ['HE images', 'HIPT_AGH_FluorescentImage_R1']
    # 2 groups to classify
    class_names = ['Responder','NonResponder']

def mkdirifNE(p):
    if not os.path.exists(p): os.mkdir(p)

mkdirifNE(PATH_ARGS.model_path)

def load_img(img_paths: list, is_mask=False):
        """ load array from a list of image paths """
        if is_mask: flag = 0
        else: flag = -1
        return np.concatenate([np.expand_dims(cv2.imread(str(img_fp), flag), axis=0)
                               for img_fp in img_paths.tolist()])
def normalize(ratios):
    """normalize a list of ratios to sum to 1"""
    return [r/sum(ratios) for r in ratios]
print(PATH_ARGS.__dict__)

{'__module__': '__main__', 'proj_path': PosixPath('/mnt/c/data/MattM_Ensemble'), 'model_path': PosixPath('/mnt/c/data/MattM_Ensemble/model_chkpts'), 'data_path': PosixPath('/mnt/c/data'), 'data_name': ['HE_RBG_Corp_images'], 'dataindex_path': PosixPath('/mnt/c/data/dataIndex.csv'), 'class_names': ['Responder', 'NonResponder'], '__dict__': <attribute '__dict__' of 'PATH_ARGS' objects>, '__weakref__': <attribute '__weakref__' of 'PATH_ARGS' objects>, '__doc__': None}


In [4]:
class preprocessor:
    """
    Preprocessing for generating tiling images from large FOV
    Generate data indexer table to be passed to dataloader
    """
    def __init__(self, data_path:Path, debug:bool=False):
        """ path to large images containing the class folders of images (non-resp , resp)"""
        self.data_path = data_path
        self.debug = debug
        # define names
        self.x_im_col = 'x_img_path'
        self.y_col = 'label'
        self.index_cols = ['parent_path','type', 'tissue', self.x_im_col]
        # 2 indexers for source/tile imgs
        self.idxer_source = dataIndexer(self.data_path/'dataIndex_source.csv', columns=self.index_cols[:-1]+[self.y_col], index_cols=self.index_cols[:-1])
        self.idxer_tile = dataIndexer(self.data_path/'dataIndex_tile.csv', columns=self.index_cols+[self.y_col], index_cols=self.index_cols)
    
    def main(self, selected_folders:list = None,
             splits={'train':0.7, 'val':0.1, 'test':0.2},
             patch_generator:str="basic",
             patch_dims:tuple=(224,224), 
             normalize_stain:bool=True
             ):
        """
        patch_generator: TIA or basic
        """
        # start indexing
        
        print(f"Folders: {selected_folders}")
        self.generate_all_tiles_all(self.data_path, 
                                    patch_generator=patch_generator,
                                    selected_folders=selected_folders, 
                                    patch_dims=patch_dims,
                                    normalize_stain=normalize_stain)
        dataindex_source = pd.read_csv(self.idxer_source.fp_csv, index_col=self.idxer_source.index_cols)
        dataindex_tile = pd.read_csv(self.idxer_tile.fp_csv, index_col=self.idxer_tile.index_cols)
        if self.debug: print(f"Indexer shape:{dataindex_tile.shape}\n columns: {dataindex_tile.columns}")
        dataindex_source = self.add_stratify_splits(dataindex_source, splits=splits)
        dataindex_tile = self.add_stratify_splits(dataindex_tile, splits=splits)
        dataindex_source.to_csv(self.idxer_source.fp_csv)
        dataindex_tile.to_csv(self.idxer_tile.fp_csv)
        print(f"[INFO] Copy of dataIndex saved to {self.data_path}")
        return dataindex_source, dataindex_tile
        
    
    def generate_all_tiles_all(self, 
                               path2_input_img_dir:Path,
                               patch_generator="basic", 
                               selected_folders:list=[], 
                               patch_dims:tuple=(224,224), 
                               normalize_stain:bool=True) ->None:
        """ 
        selected folders  list[str] 
            names of folders to be included
        """
        for dirp in path2_input_img_dir.iterdir():  # ['HE', 'HIPT']
            # dirs only + selected folders
            if dirp.is_file() or dirp.name not in selected_folders: 
                continue
            print(f"Generating tiles for data folder:{dirp}")
            #if (selected_folders is None) or (not dirp.name in selected_folders): continue
            mkdirifNE(dirp.parent/(dirp.stem+'_processed'))
            for class_dirp in dirp.iterdir(): # ['Res', 'Non-res']
                # Skip file and not selected
                if class_dirp.is_file(): continue
                # generate tiles
                print(f"[DEBUG] Directories:\nclass dirp{class_dirp}\ndirp:{dirp.parent/(dirp.name+'_processed')/class_dirp.name}")
                self.generate_all_tiles(class_dirp,
                                        save_dir_path=dirp.parent/(dirp.name+'_processed')/class_dirp.name,
                                        patch_generator=patch_generator,
                                        patch_dims=patch_dims,
                                        normalize_stain=normalize_stain)

    def generate_all_tiles(self, input_img_dir:Path, 
                           save_dir_path:Path=None,
                           patch_generator:str="basic",
                           normalize_stain: bool=True,
                           patch_dims:tuple=(224,224),
                           ) -> None:
        if save_dir_path is None:
            save_dir_path = input_img_dir/(input_img_dir.stem+"_processed")
        mkdirifNE(save_dir_path)
        # iterate (tissue) images from input_img_dir
        for N, img_fp in enumerate(input_img_dir.iterdir()):  
            print(f"Folder:{img_fp}")
            # only images files
            if not img_fp.is_file() or (img_fp.suffix!= '.png' and img_fp.suffix!= '.tif'): continue
            if self.debug: print(f"Prepare to generate tiles from {img_fp} to be saved to {save_dir_path}")
            # get tiles (generator)
            ref_img = cv2.imread(str(img_fp))  #.astype('uint8')
            print(f"Input image no.:{N} of size {ref_img.shape}")
                # tile methods
            try:
                if patch_generator=="TIA":
                    img_tiles_gen = self.tile_images_TIA(ref_img, patch_dims=patch_dims)
                else:
                    img_tiles_gen = self.tile_images_basic(ref_img, patch_dims=patch_dims)
            except Exception as e:
                print(f"Error {e} encountered, skipping {img_fp}")
            # normalize
            if normalize_stain: normalizer = self.stain_normalizer(ref_img)
            # idx source img
            self.idxer_source.write(
                        pd.DataFrame(data={
                            'parent_path':[save_dir_path], 'type':[input_img_dir.parent.stem], 
                            'tissue': [img_fp.name], self.y_col:[input_img_dir.stem]}
                        )
            )
            for tile_n, img_tile in enumerate(img_tiles_gen):                
                save_fn = save_dir_path/f"{img_fp.resolve().stem}_t{tile_n}{img_fp.resolve().suffix}"
                if self.debug: print(f"Saving tile {tile_n} to {save_fn}")
                # add to index (relative_path)
                if type(img_tile)==np.ndarray and img_tile.shape==patch_dims+(3,):
                    self.idxer_tile.write(
                        pd.DataFrame(data={
                            'parent_path':[save_fn.parent], 'type':[input_img_dir.parent.stem], 
                            'tissue': [img_fp.name], self.x_im_col:[save_fn.name], self.y_col:[input_img_dir.stem]}
                        )
                    )
                try:
                    if save_fn.is_file():   # Skip exist
                        if self.debug: print(f"\t\tSKipped...exist."); 
                        time.sleep(0.001)
                        continue
                    if normalize_stain: 
                        img_tile = normalizer.transform(img_tile)
                    # write tile
                    im = Image.fromarray(img_tile).save(str(save_fn))
                    time.sleep(0.001)
                except:
                    if self.debug: print(f"Failed for: {save_fn}")


    def tile_images_TIA(self, input_img: np.array, patch_dims=(224,224), stride=None):
        """ Tiling function from TIA 
        return generaor object """
        if not stride: stride = patch_dims
        fixed_patch_extractor = patchextraction.get_patch_extractor(
            input_img=input_img,  # input image path, numpy array, or WSI object
            # locations_list=np.array(centroids_list),
            method_name="slidingwindow",  # also supports "point" and "slidingwindow"
            patch_size=patch_dims,  # size of the patch to extract around the centroids from centroids_list
            stride=stride,  # 250 pixels overlap in both axes
            resolution=0,
            units="level",
        )
        return fixed_patch_extractor
    
    def tile_images_basic(self, im:np.array, patch_dims=(224,224)):
        """ return generator object"""
        M, N = patch_dims
        for y in range(0, im.shape[1]-N, N):
            for x in range(0,im.shape[0]-M, M):
                yield im[x:x+M, y:y+N,:]
    
    def stain_normalizer(self, target_section_image, method_name:str='Reinhard'):
        """ use entire section as input
        target_section_image: np.array
        """
        stain_normalizer = stainnorm.get_normalizer(method_name)
        stain_normalizer.fit(target_section_image)
        return stain_normalizer

    def tissue_segmenter(self, input_img_path:str=None):
        wsi = WSIReader.open(input_img=input_img_path, power=20)
        if self.debug: print(wsi.info.as_dict())
        wsi_thumb = wsi.slide_thumbnail(resolution=1.25, units="power")
        mask = wsi.tissue_mask(resolution=1.25, units="power")
        mask_thumb = mask.slide_thumbnail(
            resolution=1.25, units="power"
        )  # extracting the mask_thumb at the same resolution as wsi_thumb
        if self.debug: print(type(mask_thumb), mask_thumb.dtype, mask_thumb.shape)
        pass

    def add_stratify_splits(self, data_index:pd.DataFrame,
                            splits:dict={'train':0.7, 'val':0.1, 'test':0.2}, stratify:bool= True):
        '''stratify train test split'''
        data_index_split = data_index.copy()
        split_names = list(splits.keys())
        print(data_index_split.shape)
        for sp_name in split_names[:-1]:
            # ttsplit & stratify 
            label_col = data_index_split[self.y_col]
            (_, _, _, _, ind_train, ind_test)  = train_test_split(
                np.arange(len(label_col)), label_col, 
                label_col.index, stratify=label_col if stratify else None, test_size=1-(splits[sp_name]/sum(splits.values())))
            # set defined （'train') set in data_index 
            print(sp_name)
            data_index.loc[ind_train, 'set'] = sp_name
            # remove training set
            data_index_split = data_index_split.loc[ind_test]
            splits.pop(sp_name)
        # last type
        data_index.loc[ind_test, 'set'] = split_names[-1]
        # show group stats
        print("Group stats:\n",data_index.groupby('set')['set'].apply(lambda x: len(x)))
        return data_index

class dataIndexer:
    def __init__(self, fp_csv, columns=[], index_cols=[]):
        self.fp_csv = fp_csv
        self.index_cols = index_cols
        data_index = pd.DataFrame(dict.fromkeys(columns), index=[]).set_index(index_cols)  # all are index except y_col
        data_index.to_csv(fp_csv)
        
    def write(self, dataframe_row, on:bool=True):
        dataframe_row.set_index(self.index_cols).to_csv(self.fp_csv, mode='a', header=False)

In [5]:
# pass data folder 
print(PATH_ARGS.data_path)
prep = preprocessor(PATH_ARGS.data_path, debug=False)
print(prep.index_cols)

/mnt/c/data
['parent_path', 'type', 'tissue', 'x_img_path']


In [ ]:
data_index_df = prep.main(
    selected_folders=PATH_ARGS.data_name, splits={'train':0.7, 'val':0.1, 'test':0.2}, 
    patch_generator="basic",
    patch_dims=(224,224),
    normalize_stain=False,); 
data_index_df.head(2)

Folders: ['HE_RBG_Corp_images']
Generating tiles for data folder:/mnt/c/data/HE_RBG_Corp_images
[DEBUG] Directories:
class dirp/mnt/c/data/HE_RBG_Corp_images/.ipynb_checkpoints
dirp:/mnt/c/data/HE_RBG_Corp_images_processed/.ipynb_checkpoints
[DEBUG] Directories:
class dirp/mnt/c/data/HE_RBG_Corp_images/NonResponder
dirp:/mnt/c/data/HE_RBG_Corp_images_processed/NonResponder
Folder:/mnt/c/data/HE_RBG_Corp_images/NonResponder/.ipynb_checkpoints
Folder:/mnt/c/data/HE_RBG_Corp_images/NonResponder/NA-24_0000023786_2021-03-22 09_49_45 - Series 1 (1, x=16274, y=23991, w=3780, h=3638).tif
Input image no.:1 of size (3638, 3780, 3)
Folder:/mnt/c/data/HE_RBG_Corp_images/NonResponder/NA-24_0000023786_2021-03-22 09_49_45 - Series 1 (1, x=16466, y=14955, w=4701, h=4701).tif
Input image no.:2 of size (4701, 4701, 3)
Folder:/mnt/c/data/HE_RBG_Corp_images/NonResponder/NA-24_0000023786_2021-03-22 09_49_45 - Series 1 (1, x=20125, y=20755, w=4086, h=4063).tif
Input image no.:3 of size (4063, 4086, 3)
Folde

In [ ]:
data_index_df = pd.read_csv(PATH_ARGS.data_path/'dataIndex_source.csv', index_col=list(range(len(prep.index_cols))))
print(data_index_df.columns, data_index_df.shape)
data_index_df.head(2)

In [82]:
#data_index_df.index = data_index_df.index.set_levels(data_index_df.index.levels[0].str.replace(r'\\',r'/').str.replace(r'C:/data',r'/mnt/c/data'), level=0)
data_index_df.head(2)
data_index_df.to_csv(PATH_ARGS.data_path/(PATH_ARGS.data_name[0]+'_processed')/'dataIndex(ubuntu).csv')

In [ ]:
def check_images(data_index_df,):
    target_shape = (224, 224, 3)
    for img_fp in data_index_df.x_img_path.to_list():
        if np.asarray(Image.open(img_fp)).shape!=target_shape:
            print(f"Incorrect instance at:{img_fp}")
            break
    print("Passed.")
#check_images(data_index_df)